In [25]:
import os

In [26]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [27]:
from pyspark.sql import SQLContext, Row, SparkSession
#load the historical file and convert lines to rows
lines = sc.textFile("file:///Users/anjabelcijan/Desktop/Spark/file.txt")
words = lines.map(lambda l: l.split("\t"))

AttributeError: 'PipelinedRDD' object has no attribute 'show'

In [28]:
df = (df
             .withColumnRenamed('C0', 'Title')
             .withColumnRenamed('C1', 'Description')
             .withColumnRenamed('C2', 'Topic')
             )

In [30]:
df.printSchema()

root
 |-- Title: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Topic: string (nullable = true)



In [35]:
stopwords = sc.textFile("file:///Users/anjabelcijan/Desktop/spark/stopwords.txt").collect()

In [36]:
import re
punctuation = re.compile(r'[-.?!,":;()|0-9]')

In [37]:
def extract_tokens(sentence):
    """Tokenization of a given sentence.
       Drop stopwords, punctuations, numbers.
       Change the sentence to lowercase."""
    
    if(sentence) :
        tokens = punctuation.sub(' ', sentence.replace("'", " ")).lower().split()
        tokens_filtered = [word for word in tokens if (word not in stopwords) and (len(word) > 2)]
        return tokens_filtered
    else :
        return []

In [38]:
# Test your function
extract_tokens("Hello, World!")

['world']

In [40]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *

# Create UDF
extract_tokens_udf = udf(lambda sentence: extract_tokens(sentence), ArrayType(StringType()))

# Add tokens column to DataFrame
df_tokens = df.withColumn('tokens', extract_tokens_udf(df.Title))


In [42]:
df_tokens1 = df_tokens.withColumn('tokens2', extract_tokens_udf(df_tokens.Description))

In [44]:
df_tokens1.show(3)

+--------------------+--------------------+-------------+--------------------+------------------+
|               Title|         Description|        Topic|              tokens|           tokens2|
+--------------------+--------------------+-------------+--------------------+------------------+
|Meek Mill Wishes ...|'I wasn't feeling...|entertainment|[meek, mill, wish...|   [wasn, feeling]|
|A Victim Of Reven...|   Don't leak nudes.|entertainment|[victim, revenge,...|[don, leak, nudes]|
|Make A Pitcher Of...|     It's wine time.|entertainment|[make, pitcher, s...|      [wine, time]|
+--------------------+--------------------+-------------+--------------------+------------------+
only showing top 3 rows



In [46]:
#get the most common words: select function and the explode function on the tokens column, named "word"
from pyspark.sql.functions import explode, desc

df_words = df_tokens1.select(explode(df_tokens1.tokens).alias('word'))

In [ ]:
#not working (group by word and count the most common words)
df_words.groupBy('word').count().orderBy(desc('count')).show()

In [50]:
#find the most used sequences of 2 consecutive words in the df (using Ngram) title
from pyspark.ml.feature import NGram

bigram = NGram(n=2, inputCol='tokens', outputCol='bigrams')
df_bigram = bigram.transform(df_tokens1)

In [51]:
#repeat ngram for description
bigram = NGram(n=2, inputCol='tokens2', outputCol='bigrams2')
df_bigram2 = bigram.transform(df_bigram)

In [52]:
#not working
df_words = df_bigram.select(explode(df_bigram.bigrams).alias('word'))
df_words.groupBy('word').count().orderBy(desc('count')).show()

Py4JJavaError: An error occurred while calling o640.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 23.0 failed 1 times, most recent failure: Lost task 1.0 in stage 23.0 (TID 28, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 3 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15$$anonfun$apply$15.applyOrElse(EvaluatePython.scala:184)
	at org.apache.spark.sql.execution.python.EvaluatePython$.org$apache$spark$sql$execution$python$EvaluatePython$$nullSafeConvert(EvaluatePython.scala:208)
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15.apply(EvaluatePython.scala:180)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession.scala:747)
	at org.apache.spark.sql.SparkSession$$anonfun$6$$anonfun$apply$4.apply(SparkSession

In [58]:
#df with tokens and bigrams
df_bigram2.show(3)

+--------------------+--------------------+-------------+--------------------+------------------+--------------------+--------------------+
|               Title|         Description|        Topic|              tokens|           tokens2|             bigrams|            bigrams2|
+--------------------+--------------------+-------------+--------------------+------------------+--------------------+--------------------+
|Meek Mill Wishes ...|'I wasn't feeling...|entertainment|[meek, mill, wish...|   [wasn, feeling]|[meek mill, mill ...|      [wasn feeling]|
|A Victim Of Reven...|   Don't leak nudes.|entertainment|[victim, revenge,...|[don, leak, nudes]|[victim revenge, ...|[don leak, leak n...|
|Make A Pitcher Of...|     It's wine time.|entertainment|[make, pitcher, s...|      [wine, time]|[make pitcher, pi...|         [wine time]|
+--------------------+--------------------+-------------+--------------------+------------------+--------------------+--------------------+
only showing top 3 r

In [60]:
# trying sth else
from pyspark.ml.feature import HashingTF
hashingTF = HashingTF(inputCol="tokens", outputCol="raw1", numFeatures=30)
featurizedData = hashingTF.transform(df_bigram2)

hashingTF = HashingTF(inputCol="tokens2", outputCol="raw2", numFeatures=50)
featurized2Data = hashingTF.transform(featurizedData)

In [62]:
featurized2Data.show(3)

+--------------------+--------------------+-------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         Description|        Topic|              tokens|           tokens2|             bigrams|            bigrams2|                raw1|                raw2|
+--------------------+--------------------+-------------+--------------------+------------------+--------------------+--------------------+--------------------+--------------------+
|Meek Mill Wishes ...|'I wasn't feeling...|entertainment|[meek, mill, wish...|   [wasn, feeling]|[meek mill, mill ...|      [wasn feeling]|(30,[7,16,18,21,2...|(50,[12,29],[1.0,...|
|A Victim Of Reven...|   Don't leak nudes.|entertainment|[victim, revenge,...|[don, leak, nudes]|[victim revenge, ...|[don leak, leak n...|(30,[0,1,6,17,29]...|(50,[4,21],[2.0,1...|
|Make A Pitcher Of...|     It's wine time.|entertainment|[make, pitcher, s...|      [wine,

In [64]:
#create vector out of raw features, to use in a model
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
pair_assembler = VectorAssembler(inputCols=["raw1", "raw2"], outputCol="vector")
output = pair_assembler.transform(featurized2Data)

In [65]:
output.show(5)

+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               Title|         Description|        Topic|              tokens|             tokens2|             bigrams|            bigrams2|                raw1|                raw2|              vector|
+--------------------+--------------------+-------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Meek Mill Wishes ...|'I wasn't feeling...|entertainment|[meek, mill, wish...|     [wasn, feeling]|[meek mill, mill ...|      [wasn feeling]|(30,[7,16,18,21,2...|(50,[12,29],[1.0,...|(80,[7,16,18,21,2...|
|A Victim Of Reven...|   Don't leak nudes.|entertainment|[victim, revenge,...|  [don, leak, nudes]|[victim revenge, ...|[don leak, leak n...|(30,[0,1,6,17,29]...|(50,[4,21],[2.0,1.